# Structure checking tutorial

A complete checking analysis of a single structure follows.
use .revert_changes() at any time to recover the original structure

Structure checking is a key step before setting up a protein system for simulations. 
A number of normal issues found in structures at Protein Data Bank may compromise the success of the simulation, or may suggest that longer equilibration procedures are necessary.

The biobb_structure_checking modules allow to 
- Do basic manipulations on structures (selection of models, chains, alternative locations
- Detect and fix amide assignments, wrong chirality
- Detect and fix protein backbone issues (missing fragments, and atoms, capping)
- Detect and fix missing side-chain atoms
- Add hydrogen atoms according to several criteria
- Detect and classify clashes
- Detect possible SS bonds

biobb_structure_checking modules can used at the command line biobb_structure_checking/bin/check_structure


In [1]:
%load_ext autoreload
%autoreload 2

## Installation

#### Basic imports and initialization

In [2]:
import biobb_structure_checking
import biobb_structure_checking.constants as cts
from biobb_structure_checking.structure_checking import StructureChecking
base_dir_path=biobb_structure_checking.__path__[0]
args = cts.set_defaults(base_dir_path,{'notebook':True})


## General help

In [3]:
with open(args['commands_help_path']) as help_file:
    print(help_file.read())
#TODO: prepare a specific help method
# print_help(command)


BioBB's check_structure.py performs MDWeb structure checking set as a command line
utility.

commands:     Help on available commands
command_list: Run all tests from conf file
checkall:     Perform all checks without fixes
fixall:       Performs a default fix (v1.2)
load:         Stores structure on local cache and provide basic statistics

1. System Configuration
models [--select model_num]
    Detect/Select Models
chains [--select chain_ids]
    Detect/Select Chains
inscodes [--renum]
    Detects residues with insertion codes. No fix provided.
altloc [--select occupancy| alt_id | list of res_id:alt_id]
    Detect/Select Alternative Locations
metals [--remove All | None | Met_ids_list | Residue_list]
    Detect/Remove Metals
ligands [--remove All | None | Res_type_list | Residue_list]
    Detect/Remove Ligands
hetatm [--remove All | None | Res_type_list | Residue_list] (v1.2)
    Detect/Remove Ligands, revert modified residues
water [--remove Yes|No]
    Remove Water molecules
rem_h

Set input (PDB or local file, pdb or mmCif formats allowed) and output (local file, pdb format).  
Use pdb:pdbid for downloading structure from PDB (RCSB)

In [4]:
args['input_structure_path'] = 'pdb:6axg'
args['output_structure_path'] = '6axg_fixed.pdb'
args['debug'] = False

Initializing checking engine, loading structure and showing statistics

In [5]:

st_c = StructureChecking(base_dir_path,args)


Structure exists: 'tmpPDB/ax/6axg.cif' 
Structure pdb:6axg loaded
 PDB id: 6AXG
 Title: Structure of RasGRP4 in complex with HRas
 Experimental method: X-RAY DIFFRACTION
 Keywords: SIGNALING PROTEIN
 Resolution: 3.302 A

 Num. models: 1
 Num. chains: 12 (A: Protein, B: Protein, C: Protein, D: Protein, E: Protein, F: Protein, G: Protein, H: Protein, I: Protein, J: Protein, K: Protein, L: Protein)
 Num. residues:  2936
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  23168



#### models
Checks for the presence of models in the structure. 
MD simulations require a single structure, although some structures (e.g. biounits) may be defined as a series of models, in such case all of them are usually required.  
Use models('--select N') to select model num N for further analysis

In [6]:
st_c.models()

Running models.
1 Model(s) detected
Single model found
Running  check_only. Nothing else to do.


#### chains
Checks for chains (also obtained from print_stats), and allow to select one or more.   
MD simulations are usually performed with complete structures. However input structure may contain several copies of the system, or contains additional chains like peptides or nucleic acids that may be removed. 
Use chains('X,Y') to select chain(s) X and Y to proceed

In [7]:
st_c.chains()


Running chains.
12 Chain(s) detected
 A: Protein
 B: Protein
 C: Protein
 D: Protein
 E: Protein
 F: Protein
 G: Protein
 H: Protein
 I: Protein
 J: Protein
 K: Protein
 L: Protein
Running  check_only. Nothing else to do.


6axg have 6 copies in the crystal assimetric unit, to get a single copy, choose only A and B chains

In [8]:
args['check_only'] = False
st_c.chains('A,B')

Running chains. Options: A,B
12 Chain(s) detected
 A: Protein
 B: Protein
 C: Protein
 D: Protein
 E: Protein
 F: Protein
 G: Protein
 H: Protein
 I: Protein
 J: Protein
 K: Protein
 L: Protein
Selecting chain(s) A,B


#### altloc
Checks for the presence of residues with alternative locations. Atoms with alternative coordinates and their occupancy are reported.  
MD simulations requires a single position for each atom.  
Use altloc('occupancy | alt_ids | list of res:id) to select the alternative


In [9]:
st_c.altloc()

Running altloc.
No residues with alternative location labels detected
Running  check_only. Nothing else to do.


#### metals
Detects HETATM being metal ions allow to selectively remove them.  
To remove use metals (' All | None | metal_type list | residue list ')

In [10]:
st_c.metals()

Running metals.
No metal ions found
Running  check_only. Nothing else to do.


#### ligands
Detects HETATM (excluding Water molecules) to selectively remove them.  
To remove use ligands('All | None | Residue List (by id, by num)')


In [11]:
st_c.ligands()

Running ligands.
No ligands found
Running  check_only. Nothing else to do.


#### rem_hydrogen
Detects and remove hydrogen atoms. 
MD setup can be done with the original H atoms, however to prevent from non standard labelling, remove them is safer.  
To remove use rem_hydrogen('yes')


In [12]:
st_c.rem_hydrogen()

Running rem_hydrogen.
No residues with Hydrogen atoms found
Running  check_only. Nothing else to do.


#### water
Detects water molecules and allows to remove them
Crystallographic water molecules may be relevant for keeping the structure, however in most cases only some of them are required. These can be later added using other methods (titration) or manually.

To remove water molecules use water('yes')


In [13]:
st_c.water()

Running water.
No water molecules found
Running  check_only. Nothing else to do.


#### amide
Amide terminal atoms in Asn ang Gln residues can be labelled incorrectly.  
amide suggests possible fixes by checking the sourrounding environent.

To fix use amide ('All | None | residue_list')

Note that the inversion of amide atoms may trigger additional contacts. 

In [14]:
st_c.amide()

Running amide.
9 unusual contact(s) involving amide atoms found
 LEU A103.O   GLN A107.OE1    2.782 A
 THR A104.O   GLN A107.OE1    2.867 A
 GLU A134.OE2 GLN A223.OE1    3.066 A
 ASP A139.OD1 GLN A141.OE1    2.784 A
 ASN A248.OD1 GLU B63.O       2.760 A
 PRO A262.O   GLN A266.OE1    3.066 A
 GLN A285.OE1 ALA A344.O      2.867 A
 PRO A377.O   ASN A381.OD1    2.976 A
 ALA A391.O   GLN A395.OE1    2.601 A
Running  check_only. Nothing else to do.


Fix all amide residues and recheck

In [15]:
args['check_only'] = False
st_c.amide('all')

Running amide. Options: all
9 unusual contact(s) involving amide atoms found
 LEU A103.O   GLN A107.OE1    2.782 A
 THR A104.O   GLN A107.OE1    2.867 A
 GLU A134.OE2 GLN A223.OE1    3.066 A
 ASP A139.OD1 GLN A141.OE1    2.784 A
 ASN A248.OD1 GLU B63.O       2.760 A
 PRO A262.O   GLN A266.OE1    3.066 A
 GLN A285.OE1 ALA A344.O      2.867 A
 PRO A377.O   ASN A381.OD1    2.976 A
 ALA A391.O   GLN A395.OE1    2.601 A
Amide residues fixed all (8)
Rechecking
4 unusual contact(s) involving amide atoms found
 GLU A134.OE2 GLN A223.OE1    2.730 A
 VAL A244.O   ASN A248.OD1    2.772 A
 ASN A248.ND2 SER B65.N       3.064 A
 ASN A248.OD1 GLU B63.O       3.071 A


Comparing both checks it becomes clear that GLN A233 and ASN A248 are now in a worse situation, so should be changed back to the original labelling

In [16]:
st_c.amide('A223,A248')

Running amide. Options: A223,A248
4 unusual contact(s) involving amide atoms found
 GLU A134.OE2 GLN A223.OE1    2.730 A
 VAL A244.O   ASN A248.OD1    2.772 A
 ASN A248.ND2 SER B65.N       3.064 A
 ASN A248.OD1 GLU B63.O       3.071 A
Amide residues fixed A223,A248 (2)
Rechecking
2 unusual contact(s) involving amide atoms found
 GLU A134.OE2 GLN A223.OE1    3.066 A
 ASN A248.OD1 GLU B63.O       2.760 A


#### chiral
Side chains of Thr and Ile are chiral, incorrect atom labelling lead to the wrong chirality.  
To fix use chiral('All | None | residue_list')

In [17]:
st_c.chiral()

Running chiral.
No residues with incorrect side-chain chirality found
Running  check_only. Nothing else to do.


#### Backbone
Detects and fixes several problems with the backbone
use any of 
--fix_atoms All|None|Residue List 
--fix_chain All|None|Break list
--add_caps All|None|Terms|Breaks|Residue list
--no_recheck
--no_check_clashes


In [18]:
st_c.backbone()

Running backbone.
7 Residues with missing backbone atoms found
 SER A74    OXT
 LYS A108   OXT
 ASP A169   OXT
 GLU A430   OXT
 ASN B26    OXT
 LYS B117   OXT
 HIS B166   OXT
5 Backbone breaks found
 SER A74    - ASP A79    
 LYS A108   - ASP A112   
 ASP A169   - LEU A195   
 ASN B26    - ASP B33    
 LYS B117   - ALA B122   
No unexpected backbone links
Running  check_only. Nothing else to do.


Re-building backbone breaks with Modeller (Modeller requires a license key)

In [19]:
args['modeller_key'] = 'MODELIRANJE'
st_c.backbone('--fix_chain all')

Running backbone. Options: --fix_chain all
7 Residues with missing backbone atoms found
 SER A74    OXT
 LYS A108   OXT
 ASP A169   OXT
 GLU A430   OXT
 ASN B26    OXT
 LYS B117   OXT
 HIS B166   OXT
5 Backbone breaks found
 SER A74    - ASP A79    
 LYS A108   - ASP A112   
 ASP A169   - LEU A195   
 ASN B26    - ASP B33    
 LYS B117   - ALA B122   
No unexpected backbone links
Main chain fixes

                         MODELLER 9.22, 2019/06/19, r11413

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2019 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
              

AttributeError: 'NoneType' object has no attribute 'get_parent'

#### fixside
Detects and re-built missing protein side chains.   
To fix use fixside('All | None | residue_list')

In [20]:
st_c.fixside()

Running fixside.
11 Residues with missing side chain atoms found
 LEU A93    CG,CD1,CD2
 LEU A119   CG,CD1,CD2
 GLU A143   CG,CD,OE1,OE2
 GLN A161   CG,CD,OE1,NE2
 ARG A162   CG,CD,NE,CZ,NH1,NH2
 ARG A163   CG,CD,NE,CZ,NH1,NH2
 LEU A164   CG,CD1,CD2
 ASP A166   CG,OD1,OD2
 SER A167   OG
 GLN B150   CG,CD,OE1,NE2
 GLN B165   CG,CD,OE1,NE2
Running  check_only. Nothing else to do.


#### Add_hydrogens
 Add Hydrogen Atoms. Auto: std changes at pH 7.0. His->Hie. pH: set pH value
    list: Explicit list as [*:]HisXXHid, Interactive[_his]: Prompts for all selectable residues
    Fixes missing side chain atoms unless --no_fix_side is set
    Existing hydrogen atoms are removed before adding new ones unless --keep_h set.

In [21]:
st_c.add_hydrogen()

Running add_hydrogen.
171 Residues requiring selection on adding H atoms
CYS A65,A76,A122,A237,A298,A343,A399,B51,B80,B118
ASP A70,A79,A97,A112,A139,A166,A169,A198,A224,A272,A307,A315,A357,A367,A371,A404,A414,A420,B30,B33,B38,B47,B54,B57,B69,B92,B105,B107,B108,B119,B132,B154
GLU A63,A78,A115,A134,A143,A144,A156,A201,A204,A213,A241,A322,A325,A363,A388,A403,A419,A421,A424,A430,B3,B31,B37,B49,B62,B63,B76,B91,B98,B126,B143,B153,B162
HIS A77,A80,A89,A123,A132,A137,A199,A208,A276,A282,A299,A309,A311,A330,A354,A362,A375,A396,A407,B27,B94,B166
LYS A64,A108,A189,A190,A192,A273,A306,A318,A356,A378,B5,B16,B42,B88,B101,B104,B117,B147
ARG A101,A117,A118,A126,A131,A148,A155,A162,A163,A191,A215,A226,A235,A252,A261,A267,A280,A304,A335,A337,A338,A347,A368,A373,A385,A429,B41,B68,B73,B97,B102,B123,B128,B135,B149,B161,B164
TYR A106,A127,A211,A228,A333,A336,A383,A417,A423,A427,B4,B32,B40,B64,B71,B96,B137,B141,B157
Running  check_only. Nothing else to do.


#### getss
Detects possible -S-S- bonds based on distance criteria.
Proper simulation requires those bonds to be correctly set.

In [ ]:
st_c.getss()

#### clashes
Detects steric clashes based on distance criteria.  
Contacts are classified in: 
* Severe: Too close atoms, usually indicating superimposed structures or badly modelled regions. Should be fixed.
* Apolar: Vdw colissions.Usually fixed during the simulation.
* Polar and ionic. Usually indicate wrong side chain conformations. Usually fixed during the simulation


In [ ]:
st_c.clashes()

Complete check in a single method

In [ ]:
st_c.checkall()

In [ ]:
st_c._save_structure(args['output_structure_path'])

In [ ]:
import nglview as nv
nv.show_biopython(st_c.strucm.st[0])

